# RSI Python Implementation

In [2]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import datetime 


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




### Load data

In [3]:
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(3650)
data = pdr.get_data_yahoo('AAPL', start_date, end_date)
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-04-01,34.104286,33.250000,33.915714,33.709999,150786300.0,29.261658
2010-04-05,34.072857,33.538570,33.568573,34.070000,171126900.0,29.574146
2010-04-06,34.320000,33.857143,34.028572,34.220001,111754300.0,29.704361
2010-04-07,34.560001,34.094284,34.221428,34.371429,157125500.0,29.835800
2010-04-08,34.505714,34.005714,34.348572,34.278572,143247300.0,29.755198


In [4]:
df = data.copy()
df['delta'] = df['Adj Close'] - df['Adj Close'].shift(1)

In [7]:
def get_rsi(data, n):
    """Function that calculate RSI"""
    df = data.copy()
    df['delta'] = df['Adj Close'] - df['Adj Close'].shift(1)
    df['gain'] = np.where(df['delta'] >= 0, df['delta'], 0)
    df['loss'] = np.where(df['delta'] < 0, abs(df['delta']), 0)
    avg_gain = []
    avg_loss = []
    gain = df['gain'].tolist()
    loss = df['loss'].tolist()
    for i in range(len(df)):
        if i < n:
            avg_gain.append(np.nan)
            avg_loss.append(np.nan)
        elif i == n:
            tmp_gain = df['gain'].rolling(n).mean().tolist()[n]
            tmp_loss = df['loss'].rolling(n).mean().tolist()[n]
            avg_gain.append(tmp_gain)
            avg_loss.append(tmp_loss)
        elif i > n:
            tmp_gain = ((n-1) * avg_gain[i-1] + gain[i])/n
            tmp_loss = ((n-1) * avg_loss[i-1] + loss[i])/n
            avg_gain.append(tmp_gain)
            avg_loss.append(tmp_loss)
    df['avg_gain'] = avg_gain
    df['avg_loss'] = avg_loss
    df['RS'] = df['avg_gain'] / df['avg_loss']
    df['RSI'] = 100 - (100/(1+df['RS']))
    return df['RSI']


    

In [8]:
rsi = get_rsi(df, n = 14)

In [9]:
rsi

Date
2010-04-01          NaN
2010-04-05          NaN
2010-04-06          NaN
2010-04-07          NaN
2010-04-08          NaN
                ...    
2020-03-23    34.308996
2020-03-24    42.616119
2020-03-25    42.268336
2020-03-26    46.717011
2020-03-27    43.712796
Name: RSI, Length: 2515, dtype: float64

In [10]:
min(rsi)

nan

In [11]:
max(rsi)

nan